## Lab02
### Noisy-Channel Spelling Check
• Pdel(x|xy) = count (xy typed as x) / count(xy) = count (x|xy ) / count(xy) <br/>
• Pins(xy|x) = count(x typed as xy) / count(x) = count(xy|x) / count(x) <br/>
• Psub(y|x) = count (x typed as y) / count(x) = count(y|x) / count(x)  <br/>
• Ptrans(yx|xy) = count(xy typed as yx) / count(xy) = count(yx|xy) / count(xy) <br/>

### Estimate the probability function P(wrong, correct) as follows:<br/>
- argmaxcP(c|w) = argmaxcP(w|c)P(c)/P(w) = argmaxcP(w|c)P(c) because P(w) is constant<br/>
- Nr = distinct_number(w, c where count(w, c) = r >= 1)<br/>
- Nall = N1 + N2 + N3 + ...<br/>
- N0 = 26 ∗ 26 ∗ 26 ∗ 26 + 2 ∗ 26 ∗ 26 ∗ 26 + 26 ∗ 26 − Nall<br/>
- count1(w, c) = (r + 1) ∗ Nr+1/Nr<br/>
- if 0 <= count(w, c) = r <= 10<br/>
- Ped(w, c) = count1(w, c)/count(c)<br/>


In [1]:
import re
from collections import Counter, defaultdict
from pprint import pprint

'''Word Probability'''
def words(text): 
    return re.findall(r'\w+', text.lower())

count_word = Counter(words(open('C:/Users/asus/Downloads/nlp/big.txt').read()))
Nw = sum(count_word.values())
Pdist = {word: float(count)/Nw for word, count in count_word.items()}

def Pw(word): 
    return Pdist[word] if word in Pdist else 10/10**len(word)/Nw

In [2]:
'''Channel Probability'''
from collections import Counter
with open('C:/Users/asus/Downloads/nlp/count_1edit.txt' ,encoding='UTF-8-sig') as f:
    line = f.readlines()
    
"""Put every line in list"""
line_arr=[]
for i in range(len(line)):
     if len(line[i])>=2:
        line[i]= line[i][:-1].split('\t')
        line_arr.append(line[i])
        
"""Get key list"""
k_arr=[]
for i in range(len(line_arr)):
        k_arr.append(tuple(line_arr[i][0].split('|')))

"""Get value list"""
v_arr=[]
for i in range(len(line_arr)):
    # replace wierd words
    if line_arr[i][-1]=='' :
        print(i)
#         line_arr[i][-1] = line_arr[i][-1].replace('i|?1','1')
#     if line_arr[i][-1]=='a|?1' :
#         print(i)
#         line_arr[i][-1] = line_arr[i][-1].replace('a|?1','1')
#     if line_arr[i][-1]=='e|?1' :
#         line_arr[i][-1] = line_arr[i][-1].replace('e|?1','1')
    else:
        v_arr.append(int(line_arr[i][-1]))

"""Zip them to dict"""
word_dict = dict(zip(k_arr, v_arr))

#(w,c)= 1,2,3次的次數
Nr = (Counter(word_dict.values()))    #新增n0進去!
#w,c 的次數 
count_wc = word_dict   #新增n0進去!
# 計算每個c出現的次數統計, 產生一個新的dict儲存
count_c = dict()   
for key, value in (count_wc.items()):
    if key[1] in count_c:
        count_c[key[1]]+=value
    else:
        count_c[key[1]]=value   
Nall= sum(Nr.values())
N0 = 26*26*26*26+2*26*26*26+26*26 - Nall   #沒有出現的組合次數
Nr_smooth = [ N0 if r==0 else (r+1)*Nr[r+1] / Nr[r] for r in range(11) ]

1586


In [3]:
def Pedit(w, c):
    if (w,c) not in word_dict: #r=0
        if c in count_c:         #有在正確的list中
            return 317/N0/count_c[c]
        else:
            return 0
    elif 0<count_wc[(w,c)] <= 10:  #需要smooth的
        r = count_wc[(w,c)]
        return Nr_smooth[r]/count_c[c]
#         return (r+1)*Nr[r+1]/Nr[r]/count_c[c]  
#         (Ped(w, c) = count1(w, c)/count(c))
    else:      #>10的
        return count_wc[(w,c)]/count_c[c]   #Ped(w, c) = count1(w, c)/count(c)

In [5]:
def next_states(state):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    (L, R, edit, prob, pedit) = state
    all_word=[]

    R0, R1 = R[0:1], R[1:]
    if edit == 2:  
        return [( L + R0,     R1, edit,   Pw(L+R0+R1) , pedit)]
    else:
        all_word.append(( L + R0,     R1, edit,   Pw(L+R0+R1) , pedit))            #noedit
        all_word.append(( L ,         R1 ,edit+1, Pw(L+R1),     pedit* Pedit(L[-1:]+R0,L[-1:]) ))  #delete
        all_word.append(( L+ R1[0], R0+R1[1:], edit+1,Pw(L+ R1[0]+ R0+R1[1:]),pedit* Pedit(R0+R1[0],R1[0]+R0))
                        if len(R0)>1 else (L+R1 , R0, edit+1, Pw(L+R0+R1),    pedit*Pedit(R0+R1, R1+R0))) #transpose
        for c in letters:
            all_word.append((L + c ,    R1, edit+1, Pw(L+c+R1),    pedit*Pedit(R0, c) )) #replace
            all_word.append((L + R0 + c,R1, edit+1, Pw(L+R0+c+R1), pedit*Pedit(R0, R0+c) )) #insert      
        return all_word

In [6]:
""" small edit and large probability, 最後取機率大的 """

def correction(word):
    states = [ ("", word, 0, 0, 1) ]
    for i in range(len(word)):
        states = [ states for state in states for states in next_states(state)]
#         states = list(set(states))
#         states = sorted(states, key=lambda x: (x[2]))[:1000]
     
    temp = defaultdict(list)
    for state in states:
        L, R, edits, pw, pedit = state
        temp[L+R].append(state)  #L+R相同的選state裡edit最小的
    states = [min(substates, key=lambda x: (x[2],x[3]*x[4]*(-1))) for wd, substates in temp.items()]
    
## state[2]==0的取出 sort
    states0 = [state for state in states if states[2]==0]
    states0 = sorted(states0, key=lambda x: -1*x[3]*x[4])
    
## state[2]!=0的另外分類    
    states_ = [state for state in states if states[2]!=0]
    states_ = sorted(states_, key=lambda x: -1*x[3]*x[4])
    
    states = (states0 + states_)[:500]

    return sorted(states, key=lambda x: ((-1)*x[3]*x[4],(-1)*x[3]))[:3]

In [7]:
"""Test result"""
correction('speling')

[('speaking', '', 2, 0.00016583227633931973, 0.01929828387012364),
 ('spelling', '', 1, 3.585562731660967e-06, 0.5714285714285714),
 ('spoiling', '', 2, 6.2747347804066925e-06, 0.23109879032258063)]

In [8]:
correction('apparant')

[('apparent', '', 1, 3.764840868244015e-05, 0.2371191135734072),
 ('apparant', '', 0, 8.963906829152418e-14, 1),
 ('apparan', '', 1, 8.963906829152418e-13, 0.08217592592592593)]

In [9]:
correction('runing')

[('running', '', 1, 0.00012549469560813384, 0.9130434782608695),
 ('ruining', '', 1, 2.6891720487457255e-06, 0.8454545454545455),
 ('turning', '', 2, 0.0001864492620463703, 0.010493542435424354)]

In [10]:
correction('thenks')

[('thanks', '', 1, 3.495923663369443e-05, 0.2820030120481928),
 ('thinks', '', 1, 2.2409767072881046e-05, 0.3411458333333333),
 ('think', '', 2, 0.0004992896103837897, 0.013568300189393938)]